In [1]:
import os
from typing import Tuple

from datetime import datetime, timedelta

from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.json_loader import JSONLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint as HuggingFaceHub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.vectorstores import TimescaleVector
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
huggingfacehub_api_token = os.environ['HUGGINGFACE_API_KEY']

In [3]:
SERVICE_URL = os.environ["timescale_url"]

In [4]:
collection_name = "eer_meetings"
partition_interval = timedelta(days=7)

In [5]:
document_content_description = "The transcript of EER meetings containing, speaker name, the date of the meeting and transcript content."
metadata_field_info = [
    AttributeInfo(
        name="file_id",
        description="name of the file",
        type="uuid",
    ),
    AttributeInfo(
        name="zoom_date",
        description="The timestamp of the file. Specify in YYYY-MM-DDTHH:MM:SS format",
        type="datetime.datetime",
    ),
    AttributeInfo(
        name="speaker_name",
        description="The name of the speaker of the content",
        type="string",
    ),
]

In [6]:
embeddings = HuggingFaceEmbeddings()

c:\Users\szh\Desktop\EER\EER-chatbot-UI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
vectorstore = TimescaleVector(
    embedding = embeddings,
    collection_name = collection_name,
    service_url = SERVICE_URL,
    time_partition_interval = partition_interval,
)

In [8]:
llm = HuggingFaceHub(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",
        temperature=1.0,
        top_p=0.8,
        top_k=50,
        huggingfacehub_api_token=os.getenv('HUGGINGFACE_API_KEY')
    )

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\szh\.cache\huggingface\token
Login successful


In [9]:
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    enable_limit=True,
    verbose=True,
)

In [10]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [11]:
prompt = ChatPromptTemplate.from_template(template)

In [12]:
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
class Question(BaseModel):
    __root__: str


chain = chain.with_types(input_type=Question)

In [15]:
chain.invoke("hi")

"\nContext: Hello! How can I help you today?\n\nYour Answer: Hello! I'd be happy to help answer any question you have to the best of my ability. Please provide the question you have in mind."